# What databases need to be downloaded:
* smartsheets - API is running no need
* RTR - aDataManagerTicketExport 
* RTR URL: https://rtr.tdr.tetratech.com/rs

# # What we are checking over view:

#### Multiple checks
* Soil data with share ppoint and Smart sheet comparasions(learning curve to read the out put)
* Soil- Location in smart sheets
* Soil -Sample date after recived from lab dates
* Soil -Sample approval but debris status still in soils
#### other checks
* Soil sample prio to debris finish
* ROE checks - ROE date is blank | ROE Verified is blank | Property Type is blank
* FSO checks - FSO sent to county is not blank with ROE type full program and FSO debris complete is blank
* ASB sample sent prior to ASB assessment
* ASB results prior to ASB sample sent date
* Debris start date is after debris finish date
* Missing lat and long
* Tree check - Checks for number of days from tree felling complete till ready for TISW
* Salmon - obsolete
* 811 A&M checks
* Bird nests
* Bridge CRossing

In [1]:
import pandas as pd
import os
import glob
import numpy as np
from datetime import timedelta
import smartsheet

Variable

In [2]:
fileType_ex = ".xlsx"
fileType_csv = ".csv"
division = 'Northern Division' # will be opposite of the division we are checking. Help check for evrything NOT in Northern Division
division_rtr = 'NORTHERN DIVISION'
# grabs smartsheet data 
# central = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*" 
# south = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"

# Load in smartsheet using the API

In [3]:
# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Smart sheet APIs
# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API
Central.errors_as_exceptions(True)# Make sure we don't miss any errors

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)# Make sure we don't miss any errors

# load in Smartsheets to pandas data frame - A FUNCTION IS NEEDED
# Function to read in column titles and row vales and make out data frames
def smartsheet_loadup(sheet):
    """Runs through the smartsheets collection of values and create a Pandas data frame"""
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))

df_both = pd.concat([Central_SS,South_SS])

In [4]:
# df_both['County'].value_counts().index.to_list


# Smart Sheets dowload upload from download folder and concate

### made better above

In [5]:
# #Central
# load = glob.glob(central+fileType_ex)
# df = pd.read_excel(max(load, key=os.path.getctime), parse_dates=True)

In [6]:
# # South Central
# load2 = glob.glob(south+fileType_ex)
# df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True)

#### Concate Central and South Central

In [7]:
# df_both = pd.concat([df,df2])

In [8]:
# df_both.info(verbose=True)

# Fucntions

In [9]:
def compare(df, comp1=None, comp2=None, comp3=None):
    """Compares columns in our dataframe"""
    if comp3 == None:
        if df[comp1] == df[comp2]:
            return 1
        else:
            return 0
    else:
        if df[comp1] == df[comp2] and df[comp2] == df[comp3]:
            return 1
        else:
            return 0

        
        
def greater(df, comp1=None, comp2=None):
    """Checks for dates greater than another"""
    if df[comp1] > df[comp2]:
        return "Greater than"
    else:
        return "Less Than"

# Soil Share Point Data

In [10]:
path_soilTracker = r"C:\Users\jacque.trahan\Downloads\Soil Sampling Tracker*"
load_soil = glob.glob(path_soilTracker + fileType_csv)
soils_sharePoint = pd.read_csv(max(load_soil, key=os.path.getctime), usecols=['APN', 'APN Status', 'Field Notes', 'Tt Recommendation', 'Date Collected',
                                                                              "Date Lab Recd",
                                                                       'State Recommendation', 'State Rec Details',
                                                                        'DTSC Mercury Site','State Recommendation',
                                                                      'State Rec Details','State Rec Date','QC State Recs','ExpectedResultsDate',
                                                                      'Current Sample Event','Division', 'Recs Notes'])

# No sample date in SS, But sampled in Soil tracker

In [11]:
soils_sharePoint = soils_sharePoint[(~soils_sharePoint['Division'].isin(['Northern Division']))]

Soil_Sample_date_checks = df_both.merge(soils_sharePoint,
                                       left_on='APN_ROW Segment',
                                       right_on='APN',
                                       how='outer')

Soil_Sample_date_checks = Soil_Sample_date_checks[['APN_ROW Segment','Street #', 'Street Name', 'County', 'Structural Status', 'Debris Crew',
        'APN Status', 'Soil Sample','Date Collected']].copy()


Soil_Sample_date_checks = Soil_Sample_date_checks[(Soil_Sample_date_checks['Soil Sample'].isnull())
                                                  & (Soil_Sample_date_checks['Date Collected'].notnull())]

Soil_Sample_date_checks.set_index('APN_ROW Segment', inplace=True)
Soil_Sample_date_checks.to_excel('No Soil Sample Dates in smartsheets.xlsx')

# RTR data import

In [12]:
Replace_APNs = {'167-310-2500':'167-310-25-00','028-370-004 21285': '028-370-004_21285',
                '028-370-004 21275':'028-370-004_21275'}

In [13]:
path = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
load_rtr = glob.glob(path+fileType_ex)
df_RTR = pd.read_excel(max(load_rtr, key=os.path.getctime))

df_RTR['Zone Name'] = df_RTR['Zone Name'].str.replace("-","")
df_RTR['Zone Name'] = df_RTR['Zone Name'].str.strip()
df_RTR['Zone Name'] = df_RTR['Zone Name'].str[:3] + "-" + df_RTR['Zone Name'].str[3:6] + "-" + df_RTR['Zone Name'].str[6:]

df_RTR= df_RTR.replace({'Zone Name': Replace_APNs})
df_RTR['Service Code'] = df_RTR['Service Code'].str.strip()


# Check APN mismatches

In [14]:
# APN_check = df_RTR.copy()
# APN_check_final = APN_check.merge(df_both,
#                                  left_on='Zone Name',
#                                  right_on='APN_ROW Segment',
#                                  how='outer')

In [15]:
# APN_check_final = APN_check_final[(APN_check_final['APN_ROW Segment'] != APN_check_final['Zone Name']) &
#                (APN_check_final['GIS Zone1'].isin(['CENTRAL DIVISION']))]

# APN_check_final = APN_check_final[['APN_ROW Segment', 'Zone Name',
#                                    'GIS Zone1','GIS Zone2']].to_excel('Check APNs in RTR VS SS.xlsx')

# Check ROE dates

In [16]:
ROE_dates = df_both[['APN_ROW Segment','Street #', 'Street Name', 'County','Debris Crew', 'Structural Status',
                         'ROE Date', 'ROE Verified',
                    'Property Type', ]]
ROE_dates = ROE_dates[(ROE_dates['ROE Date'].isnull()) | 
                     (ROE_dates['ROE Verified'].isnull()) | 
                    (ROE_dates['Property Type'].isnull())]

ROE_dates.set_index('APN_ROW Segment', inplace=True)
ROE_dates.to_excel('ROE Random Checks.xlsx')

# FSO Check

In [17]:
FSO = df_both.copy()

FSO = FSO[(~FSO['FSO Sent to County'].isnull()) & (FSO['ROE Type'] == "Full Program ROE (debris + trees)") &
          (FSO['Debris FSO Complete'].isnull())]

FSO = FSO[['APN_ROW Segment','Street #', 'Street Name', 'County', 'Structural Status','ROE Type',
           'Erosion Control Complete', 'Erosion Control Waiver',
          'Debris FSO Complete', 'FSO Complete', 'FSO Sent to County']]

FSO.set_index('APN_ROW Segment', inplace=True)
FSO.to_excel('FSO Check.xlsx')

# ASB sample sent prior to ASB assessment

In [18]:
asb_sample = df_both.copy()
asb_sample = asb_sample[['APN_ROW Segment','Street #', 'Street Name', 'County',
                         'ASB Assessment', 'ASB Sample Sent']]
asb_sample = asb_sample[(asb_sample['ASB Assessment'] > asb_sample['ASB Sample Sent'])]

asb_sample.set_index('APN_ROW Segment', inplace=True)
asb_sample.to_excel('ASB sample sent prior to ASB assessment.xlsx')

# ASB results prior to ASB sample sent date


In [19]:
asb_res = df_both.copy()
asb_res = asb_res[['APN_ROW Segment','Street #', 'Street Name', 'County',
                   'ASB Sample Sent','ASB Results Received', 'ASB Results']]
asb_res = asb_res[(asb_res['ASB Sample Sent'].isna()) & (asb_res['ASB Results'].notna())]

asb_res.set_index('APN_ROW Segment', inplace=True)
asb_res.to_excel('ASB results prior to ASB sample sent date.xlsx')

# Check for Abatment property statuses:

In [20]:
abatment = df_both.copy()
abatment = abatment[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status','ROE Type', 'Debris Crew',
                     'Tree Crew',
            'Debris Start', 'Debris Finish', 'Soil Samples Approved', 'FSO Complete',
                     'Haz Trees Present?', 'Tree Felling Start','Tree Removal Complete (ISW)', 'Tree FSW',
                     'ROE Verified', 'ROE Date']]

abatment['Abatment Expires'] = pd.to_datetime(abatment['ROE Date']).dt.date + timedelta(days=30)
abatment['Days Left'] = pd.to_datetime('today').date() - pd.to_datetime(abatment['Abatment Expires']).dt.date 

In [21]:


abatment.set_index('APN_ROW Segment',inplace=True)
abatment = abatment[(abatment['ROE Verified'] == 'Abatement') & (~abatment['Structural Status'].isin(['Ineligible']))
                    & (abatment['FSO Complete'].isnull())].to_excel('Abatment Checks.xlsx')

# Debris start after to debris finish
### Check Debris dates

In [22]:
Debris_dates = df_both.copy()
Debris_dates = Debris_dates[['APN_ROW Segment','Street #', 'Street Name', 'County',
                          'Debris Start', 'Debris Finish', 'State Rep Sign-Off for DR']]

Debris_dates['Debris Start-new'] = pd.to_datetime(Debris_dates['Debris Start'])
Debris_dates['Debris Finish-new'] = pd.to_datetime(Debris_dates['Debris Finish'])

Debris_dates.insert(Debris_dates.columns.get_loc('State Rep Sign-Off for DR')+1, "Finish start before Debris end",
                   Debris_dates.apply(greater,comp1='Debris Start-new', comp2='Debris Finish-new', axis=1))


Debris_dates['Debris Start'] = pd.to_datetime(Debris_dates['Debris Start']).dt.date.fillna(0)
Debris_dates['Debris Finish'] = pd.to_datetime(Debris_dates['Debris Finish']).dt.date.fillna(0)
Debris_dates['State Rep Sign-Off for DR'] = pd.to_datetime(Debris_dates['State Rep Sign-Off for DR']).dt.date.fillna(0)



Debris_dates.insert(Debris_dates.columns.get_loc('Finish start before Debris end')+1, 'Match debris end and state rep',
                   Debris_dates.apply(compare, comp1='Debris Finish', comp2='State Rep Sign-Off for DR', axis=1))


Debris_dates = Debris_dates[['APN_ROW Segment','Street #', 'Street Name', 'County',
                          'Debris Start', 'Debris Finish', 'State Rep Sign-Off for DR',"Finish start before Debris end",  'Match debris end and state rep']]

Debris_dates.set_index('APN_ROW Segment', inplace=True)
Debris_dates.to_excel('Check debris start finish and state sign off dates.xlsx')

# Soil Checks

## Soil sample prior to debris finish

In [23]:
soil_debris = df_both.copy()
soil_debris = soil_debris[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status',
                           'Soil Sample', 'Debris Finish']]

soil_debris = soil_debris[(soil_debris['Soil Sample'] < soil_debris['Debris Finish'])]

soil_debris.set_index('APN_ROW Segment', inplace=True)
soil_debris.to_excel("Soil prior to debris Finish.xlsx")

# long and lats missing

In [24]:
lon_lat = df_both.copy()
lon_lat = lon_lat[['APN_ROW Segment','Street #', 'Street Name', 'County','Latitude', 'Longitude']]

lon_lat = lon_lat[lon_lat['Latitude'].isna() | lon_lat['Longitude'].isna()]

lon_lat.set_index('APN_ROW Segment', inplace=True)
lon_lat.to_excel("Missing long and lat.xlsx")

# Check Tree removal and TISW dates
https://docs.google.com/spreadsheets/d/1wF0GLNJG0dKTnkT0fEuC-kAIU4pzrr_IyhLnJ5iEZ7s/edit#gid=1185136840

In [25]:
# Planning_sheet_id = '1wF0GLNJG0dKTnkT0fEuC-kAIU4pzrr_IyhLnJ5iEZ7s'
# Planning_sheet_name = '1185136840'

# #gid # in URL lets you pick the sheet you want. Will need to look for that.
# url_planning = f"https://docs.google.com/spreadsheets/d/{Planning_sheet_id}/export?gid={Planning_sheet_name}&format=csv"
# planning_google = pd.read_csv(url_planning)


# Check all dates are there for Trees

In [26]:
Tree_date_checks = df_both.copy()

Tree_date_checks = Tree_date_checks[(Tree_date_checks['Tree Felling Complete'].isnull() |
                                     Tree_date_checks['Tree Removal Complete (ISW)'].isnull()) 
                                    & (Tree_date_checks['Tree FSW'].notnull())]

Tree_date_checks = Tree_date_checks[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status',
                                    'Tree Felling Complete','Tree Removal Complete (ISW)','Tree FSW']]
Tree_date_checks.set_index('APN_ROW Segment', inplace=True)
Tree_date_checks.to_excel('Check Tree missing dates.xlsx')

# check holds and their status

In [27]:
Holds = df_both.copy()
Holds = Holds[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status','ROE Type',
               'Debris Crew', 'Tree Crew',
            'Debris Start', 'Debris Finish',
               'Soil Samples Approved', 'FSO Complete', 'FSO Sent to County',
                     'Tree FSW','Hold', 'Hold Reason',
                     'ROE Verified']]

Holds = Holds[Holds['Hold'] == True].to_excel("Holds checks.xlsx")

# Check Bird Nests that are active

In [28]:
birds = df_both.copy()
birds = birds[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status',
              'Nesting Bird Survey Date', 'Nesting Bird Survey Results', 'Hold']]
birds = birds[(birds['Nesting Bird Survey Results'] == "Active Nest") & (birds['Hold'] != 1.0) &
             (~birds['Structural Status'].isin(['Returned to County', 'Ready for Final Sign-off', 'Ready for Erosion Control']))]

birds.set_index('APN_ROW Segment', inplace=True)
birds.to_excel('Active Birds nest not on hold.xlsx')

# bridge/crossing checks:

In [29]:
bridge = df_both.copy()
bridge = bridge[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status',
                'OES EHP Attention Required',
                 'Water crossing/bridge needed?', 'Water crossing/bridge install', 'Water crossing/bridge removal']]

bridge = bridge[bridge['OES EHP Attention Required'].notna()]
bridge.set_index('APN_ROW Segment', inplace=True)
bridge.to_excel('Check for bridge updates.xlsx')

# Check pending vehicles

In [30]:
vecs = df_both.copy()
vecs = vecs[['APN_ROW Segment','Street #', 'Street Name', 'County','Structural Status', 'Debris Crew',
            'Vehicle Removal Complete']]
# conditions
vecs = vecs[(vecs['Debris Crew'] == 'FSO Pending Vehicles')]
vecs.to_excel('Check FSO pending vecs and if vecs were removed.xlsx')

# Make sure Fire Names are not missing:

In [31]:
fire_name = df_both[['APN_ROW Segment','Street #', 'Street Name', 'County','Fire Name']]
fire_name = fire_name[fire_name['Fire Name'].isnull()]
fire_name.to_excel('Firenames.xlsx')

# Checks for missing Special Notes

In [32]:
notes = df_both[(df_both['ROE Type'] == 'Full Program ROE (debris + trees)') & 
               (df_both['Debris Finish'].notnull())]

notes = notes[['APN_ROW Segment','Street #', 'Street Name', 'County', 'ROE Type', 'Structural Status',
                          'Debris Start', 'Debris Finish', 'State Rep Sign-Off for DR', 'Special Notes']]
notes.to_excel('get missing Special notes.xlsx')